In [3]:
import pandas as pd

def normalize_email(email):
    """
    Normalize email addresses by replacing "googlemail.com" with "gmail.com"
    and converting to lowercase for case-insensitive comparison.
    """
    if pd.isna(email):
        return email
    email_lower = email.lower()
    if "googlemail.com" in email_lower:
        return email_lower.replace("googlemail.com", "gmail.com")
    return email_lower

# 1. Read Google.contacts.xlsx
google_contacts = pd.read_excel(
    'Google.contacts.xlsx',
    sheet_name='Sheet1',
    usecols=['Name', 'Given Name', 'Additional Name', 'Family Name', 'E-mail 1 - Value', 'E-mail 2 - Value', 'Phone 1 - Value'],
    engine='openpyxl'
)

# 2. Read MPGS.contacts.xlsx
mpgs_contacts = pd.read_excel(
    'MPGS.contacts.xlsx',
    sheet_name='Sheet1',
    usecols=['Name', 'Vorname', 'Adresse', 'PLZ', 'Ort', 'Geb. Datum', 'Telefon', 'E-Mail'],
    engine='openpyxl'
)

# Normalize email addresses in both dataframes
google_contacts['E-mail 1 - Value'] = google_contacts['E-mail 1 - Value'].apply(normalize_email)
google_contacts['E-mail 2 - Value'] = google_contacts['E-mail 2 - Value'].apply(normalize_email)
mpgs_contacts['E-Mail'] = mpgs_contacts['E-Mail'].apply(normalize_email)

# 3. Find all entries from "mpgs contacts" that have "E-Mail" set and where this eMail address is not contained in "google contacts"
mask = (
    mpgs_contacts['E-Mail'].notna() &
    ~mpgs_contacts['E-Mail'].isin(google_contacts['E-mail 1 - Value']) &
    ~mpgs_contacts['E-Mail'].isin(google_contacts['E-mail 2 - Value'])
)

# 4. Write these entries from "mpgs contacts" to a new dataframe "missing contacts"
missing_contacts = mpgs_contacts[mask]


# 5. Find all entries from "google contacts" where neither 'E-mail 1 - Value' nor 'E-mail 2 - Value' is in "mpgs contacts" 'E-Mail'
mask_updatable = (
    (~google_contacts['E-mail 1 - Value'].isin(mpgs_contacts['E-Mail']) &
    ~google_contacts['E-mail 2 - Value'].isin(mpgs_contacts['E-Mail'])) &
    (google_contacts['E-mail 1 - Value'].notna() | google_contacts['E-mail 2 - Value'].notna())
)

# 6. Write these entries from "google contacts" to a new dataframe "updatable contacts"
updatable_contacts = google_contacts[mask_updatable]

# If you want to save "missing contacts" to an Excel file, you can do so like this:
missing_contacts.to_excel('missing_contacts.xlsx', index=False, engine='openpyxl')
updatable_contacts.to_excel('updatable_contacts.xlsx', index=False, engine='openpyxl')

